<a href="https://colab.research.google.com/github/akshay-r13/ds_from_scratch/blob/main/17_Decision_Trees.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 17 - Decision Trees

## What is a Decision Tree?

A decision tree uses a tree structure to represent a number of possible decision paths and the corresponding outcome for each path.

In [1]:
import math

In [2]:

# Function to compute entropy

def entropy(class_probabilities):
  return sum( -p * math.log(p, 2) 
              for p in class_probabilities
              if p != 0 # to avoid calculating log(0) = infinity
             )

In [3]:
entropy([0, 0, 0, 1])

0.0

In [4]:
from collections import Counter

In [5]:
# Function to compute class probabilities

def class_probabilities(labels):
  total_count = len(labels) # Total count of labels
  return [label_count / total_count for label_count in Counter(labels).values()]

In [6]:
class_probabilities(['a', 'a', 'a', 'b'])

[0.75, 0.25]

In [7]:
def data_entropy(labelled_data):
  labels = [label for _, label in labelled_data]
  class_probs = class_probabilities(labels)
  return entropy(class_probs)


In [8]:
# Function to compute partition entropy

def partition_entropy(subsets):
  total_count = sum(len(subset) for subset in subsets)

  return sum( (len(subset) / total_count) * data_entropy(subset) # compute weighted entropy
              for subset in subsets) # Iterate each subset
    

## Creating a Decision Tree

In [9]:
inputs = [
  ({'level':'Senior', 'lang':'Java', 'tweets':'no', 'phd':'no'},
  False),
  ({'level':'Senior', 'lang':'Java', 'tweets':'no', 'phd':'yes'},
  False),
  ({'level':'Mid', 'lang':'Python', 'tweets':'no', 'phd':'no'},
  True),
  ({'level':'Junior', 'lang':'Python', 'tweets':'no', 'phd':'no'},
  True),
  ({'level':'Junior', 'lang':'R', 'tweets':'yes', 'phd':'no'},
  True),
  ({'level':'Junior', 'lang':'R', 'tweets':'yes', 'phd':'yes'},
  False),
  ({'level':'Mid', 'lang':'R', 'tweets':'yes', 'phd':'yes'},
  True),
  ({'level':'Senior', 'lang':'Python', 'tweets':'no', 'phd':'no'}, False),
  ({'level':'Senior', 'lang':'R', 'tweets':'yes', 'phd':'no'},
  True),
  ({'level':'Junior', 'lang':'Python', 'tweets':'yes', 'phd':'no'}, True),
  ({'level':'Senior', 'lang':'Python', 'tweets':'yes', 'phd':'yes'}, True),
  ({'level':'Mid', 'lang':'Python', 'tweets':'no', 'phd':'yes'},
  True),
  ({'level':'Mid', 'lang':'Java', 'tweets':'yes', 'phd':'no'},
  True),
  ({'level':'Junior', 'lang':'Python', 'tweets':'no', 'phd':'yes'}, False)
]

In [10]:
from collections import defaultdict

In [11]:
# Function to partition data by an attribute

def partition_by(inputs, attribute):
  groups = defaultdict(list)
  for input in inputs:
    groups[input[0][attribute]].append(input)
  return groups

In [12]:
# Function to compute partition entropy based on attribute

def partition_entropy_by(inputs, attribute):
  partitions = partition_by(inputs, attribute)
  return partition_entropy(partitions.values())

In [13]:
partition_entropy_by(inputs, 'level')

0.6935361388961918

In [14]:
for key in ['level', 'lang', 'phd', 'tweets']:
  print(key, partition_entropy_by(inputs, key))

level 0.6935361388961918
lang 0.8601317128547441
phd 0.8921589282623617
tweets 0.7884504573082896


In [15]:
partition_by(inputs, 'level')['Mid']

[({'lang': 'Python', 'level': 'Mid', 'phd': 'no', 'tweets': 'no'}, True),
 ({'lang': 'R', 'level': 'Mid', 'phd': 'yes', 'tweets': 'yes'}, True),
 ({'lang': 'Python', 'level': 'Mid', 'phd': 'yes', 'tweets': 'no'}, True),
 ({'lang': 'Java', 'level': 'Mid', 'phd': 'no', 'tweets': 'yes'}, True)]

In [16]:
senior_inputs =  partition_by(inputs, 'level')['Senior']
print(senior_inputs)

[({'level': 'Senior', 'lang': 'Java', 'tweets': 'no', 'phd': 'no'}, False), ({'level': 'Senior', 'lang': 'Java', 'tweets': 'no', 'phd': 'yes'}, False), ({'level': 'Senior', 'lang': 'Python', 'tweets': 'no', 'phd': 'no'}, False), ({'level': 'Senior', 'lang': 'R', 'tweets': 'yes', 'phd': 'no'}, True), ({'level': 'Senior', 'lang': 'Python', 'tweets': 'yes', 'phd': 'yes'}, True)]


In [17]:
for key in ['level', 'lang', 'phd', 'tweets']:
  print(key, partition_entropy_by(senior_inputs, key))

level 0.9709505944546686
lang 0.4
phd 0.9509775004326937
tweets 0.0


## Putting it all together

In [18]:
inputs

[({'lang': 'Java', 'level': 'Senior', 'phd': 'no', 'tweets': 'no'}, False),
 ({'lang': 'Java', 'level': 'Senior', 'phd': 'yes', 'tweets': 'no'}, False),
 ({'lang': 'Python', 'level': 'Mid', 'phd': 'no', 'tweets': 'no'}, True),
 ({'lang': 'Python', 'level': 'Junior', 'phd': 'no', 'tweets': 'no'}, True),
 ({'lang': 'R', 'level': 'Junior', 'phd': 'no', 'tweets': 'yes'}, True),
 ({'lang': 'R', 'level': 'Junior', 'phd': 'yes', 'tweets': 'yes'}, False),
 ({'lang': 'R', 'level': 'Mid', 'phd': 'yes', 'tweets': 'yes'}, True),
 ({'lang': 'Python', 'level': 'Senior', 'phd': 'no', 'tweets': 'no'}, False),
 ({'lang': 'R', 'level': 'Senior', 'phd': 'no', 'tweets': 'yes'}, True),
 ({'lang': 'Python', 'level': 'Junior', 'phd': 'no', 'tweets': 'yes'}, True),
 ({'lang': 'Python', 'level': 'Senior', 'phd': 'yes', 'tweets': 'yes'}, True),
 ({'lang': 'Python', 'level': 'Mid', 'phd': 'yes', 'tweets': 'no'}, True),
 ({'lang': 'Java', 'level': 'Mid', 'phd': 'no', 'tweets': 'yes'}, True),
 ({'lang': 'Python', 

### ID3 Algorithm

1. If all labels in a partition are the same, create a leaf node predicting the label and finish.
2. If there are no more features to split over, create a leaf node predicting the most common label in the partition and finish.
3. Else split the data into partitions based on the features
4. Compute entropy for each partition.
5. Pick the feature which produces a partition with the least entropy
6. Recursively create a subtree by splitting data into the chosen feature

In [19]:
partition_entropy_by(inputs, 'lang')

0.8601317128547441

In [20]:
def build_tree_id3(inputs, split_candidates=None):

  # split candidates are the features over which the data should be split.
  # In the first run, it would be 'None' and data should be split over all the features
  if split_candidates == None:
    split_candidates = inputs[0][0].keys()
  
  # Count trues and falses in the input
  num_inputs = len(inputs)
  num_trues = len([input for input in inputs if input[1] == True])
  num_falses = num_inputs - num_trues

  if num_trues == 0: return False # If all are False return False
  if num_falses == 0: return True # If all are True return True

  if not split_candidates: # if no split candidates left
    return num_trues >= num_falses # return most common

  # Compute entropy for each split candidate
  entropy_by_partition =  {attr:partition_entropy_by(inputs, attr) for attr in split_candidates}
  # Pick the feature with least entropy
  # print(entropy_by_partition)
  best_attribute = min(entropy_by_partition, key=lambda k: entropy_by_partition[k])
  # print(best_attribute)
  # Partition based on best attribute
  partitions = partition_by(inputs, best_attribute)

  # create new candidates excluding the best attribute
  new_candidates = [cand for cand in split_candidates if cand != best_attribute]

  # split over the best attribute
  subtrees = {attr_value: build_tree_id3(subset, new_candidates) for attr_value, subset in partitions.items()}

  # default case to handle when subtree not available for feature
  subtrees[None] = num_trues > num_falses

  return (best_attribute, subtrees)




In [21]:
tree = build_tree_id3(inputs)
tree

('level',
 {'Junior': ('phd', {None: True, 'no': True, 'yes': False}),
  'Mid': True,
  None: True,
  'Senior': ('tweets', {None: False, 'no': False, 'yes': True})})

In [22]:
# A function to classify based on the input
def classify(tree, input):
  # If leaf node, return value
  if tree in [True, False]:
    return tree

  # Otherwise it is a tree where
  # key = attribute on which node decision is mode
  # value = subtree under the node

  attribute, subtrees = tree

  # to handle cases where attribute is not in input
  subtree_key = input.get(attribute)
  if subtree_key not in subtrees.keys():
    subtree_key = None

  # pick the corresponding subtree using the attribute value
  subtree = subtrees[subtree_key]

  return classify(subtree, input)



In [23]:
classify(tree, { "level" : "Junior",
                 "lang" : "Java",
                 "tweets" : "yes",
                 "phd" : "no"} )        # True

True

In [24]:
classify(tree, { "level" : "Junior",
                 "lang" : "Java",
                 "tweets" : "yes",
                 "phd" : "yes"} )       # False

False

In [25]:
print(classify(tree, { "level" : "Intern" } )) # True
print(classify(tree, { "level" : "Senior" } )) # False

True
False


In [26]:
partition_by(inputs, 'lang')

defaultdict(list,
            {'Java': [({'lang': 'Java',
                'level': 'Senior',
                'phd': 'no',
                'tweets': 'no'},
               False),
              ({'lang': 'Java',
                'level': 'Senior',
                'phd': 'yes',
                'tweets': 'no'},
               False),
              ({'lang': 'Java', 'level': 'Mid', 'phd': 'no', 'tweets': 'yes'},
               True)],
             'Python': [({'lang': 'Python',
                'level': 'Mid',
                'phd': 'no',
                'tweets': 'no'},
               True),
              ({'lang': 'Python',
                'level': 'Junior',
                'phd': 'no',
                'tweets': 'no'},
               True),
              ({'lang': 'Python',
                'level': 'Senior',
                'phd': 'no',
                'tweets': 'no'},
               False),
              ({'lang': 'Python',
                'level': 'Junior',
                'phd': 'no',
  